In [1]:
import numpy as np
import sys
sys.path.append('../../pysimARG')
from clonal_genealogy import ClonalTree
from ClonalOrigin_nodes import ClonalOrigin_nodes
import ClonalOrigin_pair
from tree import tree

## Check inputs

In [2]:
tree1 = ClonalTree(n=5)

In [3]:
print(tree1)

ClonalTree(n=5)


In [4]:
tree1.edge

array([[6.        , 1.        , 0.06785301],
       [6.        , 5.        , 0.06785301],
       [7.        , 4.        , 0.16877495],
       [7.        , 3.        , 0.16877495],
       [8.        , 6.        , 0.21696331],
       [8.        , 7.        , 0.38573826],
       [9.        , 8.        , 0.41625858],
       [9.        , 2.        , 0.41625858]])

In [5]:
tree1.node_height

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.06785301, 0.16877495, 0.38573826, 0.41625858])

## Fix inputs

In [6]:
n = 5
clonal_edge = np.array([
    [6,     1, 0.1973661],
    [6,     5, 0.1973661],
    [7,     4, 0.3382209],
    [7,     6, 0.1408549],
    [8,     2, 0.5565577],
    [8,     7, 0.2183368],
    [9,     8, 0.6343786],
    [9,     3, 1.1909363]
])
clonal_node_height = np.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.1973661, 0.3382209, 0.5565577, 1.1909363])

In [7]:
rho_site = 0.5
L = 100
rho = L * rho_site
delta = 5.0
k = 20

## Check part 1

In [83]:
# Tree length
tree_length = np.sum(clonal_edge[:, 2])

# Initialize recombination edges
nrow_max = 1000
# Columns: b_edge, b_height, a_edge, a_height, x, y
recomb_edge = np.full((nrow_max, 6), np.nan)

# Add recombination sequentially
n_recomb = 0
remain_index = np.array([], dtype=int)

In [ ]:
for i in range(1, 3):  # i = 1, 2
    if i == 1:
        R_new = np.random.poisson(rho_site * delta * tree_length / 2)
        R_old = 0
    else:  # i == 2
        survive_index = np.where(recomb_edge[:n_recomb, 5] == 1)[0] if n_recomb > 0 else np.array([], dtype=int)
        delta2 = np.sum((1 - 1/delta) ** np.arange(k))
        R_new = np.random.poisson(rho_site * delta2 * tree_length / 2)
        
        if len(survive_index) >= 0:
            R_old = np.random.binomial(len(survive_index), (1 - 1/delta) ** k)
            if len(survive_index) == 1:
                remain_index = survive_index.copy()
            elif len(survive_index) > 1 and R_old > 0:
                remain_index = np.random.choice(survive_index, R_old, replace=False)
            else:
                remain_index = np.array([], dtype=int)
        else:
            R_old = 0
    
    if R_new > 0:
        # Expand matrix if needed
        if n_recomb + R_new >= nrow_max:
            recomb_edge = np.vstack([recomb_edge, np.full((nrow_max, 6), np.nan)])
            nrow_max = 2 * nrow_max
        
        # Set x and y columns
        recomb_edge[n_recomb:n_recomb + R_new, 4] = i  # x
        recomb_edge[n_recomb:n_recomb + R_new, 5] = i  # y
        
        a_rexp = np.random.exponential(1.0, size=R_new)
        
        # Simulate b_edge (similar to mutation)
        # Sample edges with probability proportional to edge length
        edge_probs = clonal_edge[:, 2] / np.sum(clonal_edge[:, 2])
        recomb_edge[n_recomb:n_recomb + R_new, 0] = np.random.choice(
            range(1, (2*n-1)), R_new, replace=True, p=edge_probs
        )
        
        for j in range(R_new):
            idx = n_recomb + j
            b_edge_idx = int(recomb_edge[idx, 0]) - 1
            
            # Simulate b_height
            recomb_edge[idx, 1] = (
                np.random.uniform(0, clonal_edge[b_edge_idx, 2]) +
                clonal_node_height[int(clonal_edge[b_edge_idx, 1])-1]
            )
            
            # Identify a_height
            # t_above_b: heights of internal nodes minus b_height
            t_above_b = clonal_node_height[n:2*n-1] - recomb_edge[idx, 1]
            
            # Get positive values (nodes above b)
            positive_mask = t_above_b >= 0
            positive_t = t_above_b[positive_mask]
            
            # i_above_b with 0 prepended
            i_above_b_full = np.concatenate([[0], np.sort(positive_t)])
            i_above_b = np.diff(i_above_b_full)
            
            # Calculate cumulative values
            num_intervals = len(i_above_b)
            lineage_counts = np.arange(num_intervals + 1, 1, -1)
            cuml_above_b = np.cumsum(i_above_b * lineage_counts)
            
            # Determine number of lineages at coalescence time
            num_lineage = (num_intervals + 1) - np.sum(a_rexp[j] > cuml_above_b)
            
            if num_lineage == (num_intervals + 1):
                recomb_edge[idx, 3] = a_rexp[j] / num_lineage + recomb_edge[idx, 1]
            else:
                idx_cuml = num_intervals - num_lineage
                recomb_edge[idx, 3] = (
                    (a_rexp[j] - cuml_above_b[idx_cuml]) / num_lineage +
                    np.sum(i_above_b[:idx_cuml + 1]) +
                    recomb_edge[idx, 1]
                )
            
            # Simulate a_edge
            if num_lineage > 1:
                a_height = recomb_edge[idx, 3]
                # Find edges that span the a_height
                pool_edge = np.where(
                    (clonal_node_height[clonal_edge[:, 0].astype(int)-1] >= a_height) &
                    (clonal_node_height[clonal_edge[:, 1].astype(int)-1] < a_height)
                )[0] + 1
                recomb_edge[idx, 2] = np.random.choice(pool_edge)
            else:
                # Root edge (using edge index 2*n-2 for 0-indexed)
                recomb_edge[idx, 2] = 2 * n - 1
    
    if R_old > 0 and len(remain_index) > 0:
        recomb_edge[remain_index, 5] = i
    
    n_recomb = n_recomb + R_new

# Handle case with no recombination
if n_recomb == 0:
    edge = clonal_edge
    edge_mat = np.full((2 * (n - 1), 2), True)
    node_height = clonal_node_height
    node_mat = np.full((2 * n - 1, 2), True)
    node_clonal = np.full(2 * n - 1, True)
    sum_time = np.max(clonal_node_height)

# Trim recomb_edge to actual size
recomb_edge = recomb_edge[:n_recomb, :]

In [99]:
n_recomb

17

In [100]:
recomb_edge.shape

(17, 6)

## Fix part 2 input

In [2]:
recomb_edge = np.array([
    [8, 0.394919897,      9, 2.33454667, 1, 1],
    [4, 0.265094688,      7, 0.89917565, 1, 1],
    [8, 0.844189355,      7, 0.99794819, 1, 1],
    [8, 0.868737984,      9, 2.18565870, 1, 1],
    [6, 0.356841524,      6, 0.37098054, 1, 1],
    [8, 1.178990918,      9, 1.92812297, 1, 1],
    [8, 0.418750008,      6, 0.45349136, 1, 1],
    [6, 0.352237455,      7, 0.77101343, 1, 1],
    [8, 0.050975039,      2, 0.06274333, 1, 1],
    [8, 0.008767799,      6, 0.36282610, 1, 1],
    [8, 0.055685627,      8, 0.12471992, 1, 1],
    [5, 0.093964265,      1, 0.11169998, 1, 1],
    [1, 0.080062175,      8, 0.78815058, 1, 1],
    [7, 0.827924395,      7, 0.92772200, 2, 2],
    [7, 1.182302840,      9, 2.94700281, 2, 2],
    [6, 0.391514106,      5, 0.43231363, 2, 2],
    [5, 0.081080721,      2, 0.15545495, 2, 2],
    [8, 0.184268881,      3, 0.23812102, 2, 2],
    [7, 0.751048207,      9, 2.74427834, 2, 2]
])

In [3]:
recomb_edge.shape

(19, 6)

In [4]:
n_recomb = 19

## Check part 2 and `ClonalOrigin_nodes`

check `node_max`, `edge_max`, `edge_matrix`, `edge_mat_index`, `node_mat`, `node_info`, `recomb_node`

In [69]:
# Build full ARG
node_max = 2 * n - 1 + 3 * n_recomb
edge_max = 2 * (n - 1) + 4 * n_recomb

edge_matrix = np.full((edge_max, 3), np.nan)
edge_mat_index = np.full(edge_max, np.nan)
node_mat = np.full((node_max, 2), np.nan)
node_info = np.full((node_max, 4), np.nan)
# Columns: index, height, recomb, clonal

node_mat[:n, :] = True
node_info[:, 0] = np.arange(1, node_max + 1)

# Set clonal node info
node_info[:2*n-1, 1] = clonal_node_height
node_info[:2*n-1, 2] = 0
node_info[:2*n-1, 3] = 1  # True -> 1

# Set recombination out nodes (b nodes)
# Interleave: for each recomb event, add two nodes at b_height
for r in range(n_recomb):
    base_idx = 2 * n - 1 + 2 * r
    node_info[base_idx, 1] = recomb_edge[r, 1]      # b_height
    node_info[base_idx, 2] = -(r + 1)               # negative recomb index (1-indexed)
    node_info[base_idx, 3] = 1                      # clonal = True
    
    node_info[base_idx + 1, 1] = recomb_edge[r, 1]  # same b_height
    node_info[base_idx + 1, 2] = np.nan             # NA
    node_info[base_idx + 1, 3] = 0                  # clonal = False

# Set recombination in nodes (a nodes)
for r in range(n_recomb):
    idx = 2 * n - 1 + 2 * n_recomb + r
    node_info[idx, 1] = recomb_edge[r, 3]           # a_height
    node_info[idx, 2] = r + 1                       # positive recomb index (1-indexed)
    node_info[idx, 3] = 1                           # clonal = True

In [70]:
# Sort by height
sort_order = np.lexsort((node_info[:, 0], node_info[:, 1]))
node_info = node_info[sort_order, :]

# Recombination nodes on every edge
# Use 1-indexed edge indices to match R behavior
recomb_node = []
for edge_idx in range(1, 2 * n):
    # Convert to 1-indexed for ClonalOrigin_nodes
    nodes = ClonalOrigin_nodes(recomb_edge, edge_idx)
    recomb_node.append(nodes)

In [34]:
node_max, edge_max

(66, 84)

In [35]:
edge_matrix.shape

(84, 3)

In [36]:
edge_mat_index.shape

(84,)

In [37]:
node_mat[:10, :]

array([[ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [ 1.,  1.],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]])

In [38]:
node_mat.shape

(66, 2)

In [39]:
node_info[:, 3]

array([1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1.,
       0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1.,
       1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0.,
       1., 0., 1., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1.])

In [40]:
node_info.shape

(66, 4)

In [41]:
node_info

array([[ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 2.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 3.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 4.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 5.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00],
       [ 2.80000000e+01,  8.76779900e-03, -1.00000000e+01,
         1.00000000e+00],
       [ 2.90000000e+01,  8.76779900e-03,             nan,
         0.00000000e+00],
       [ 2.60000000e+01,  5.09750390e-02, -9.00000000e+00,
         1.00000000e+00],
       [ 2.70000000e+01,  5.09750390e-02,             nan,
         0.00000000e+00],
       [ 3.00000000e+01,  5.56856270e-02, -1.10000000e+01,
         1.00000000e+00],
       [ 3.10000000e+01,  5.56856270e-02,             nan,
         0.00000000e+00],
       [ 5.60000000e+01,  6.27433300e-02,  9.00000000e+00,
      

In [18]:
len(recomb_node)

9

In [42]:
recomb_node

[array([-13.,  12.]),
 array([ 9., 17.]),
 array([18.]),
 array([-2.]),
 array([-17., -12.,  16.]),
 array([ -8.,  -5.,  10.,   5., -16.,   7.]),
 array([-19.,   8., -14.,   2.,  14.,   3., -15.]),
 array([-10.,  -9., -11.,  11., -18.,  -1.,  -7.,  13.,  -3.,  -4.,  -6.]),
 array([ 6.,  4.,  1., 19., 15.])]

## Fix part 3 input
set `node_info`, `clonal_edge`, `recomb_node`, `recomb_edge`

## Check part 3
check `edge_matrix`, `node_mat`, `edge_mat_index`, `node_info`

In [59]:
recomb_val = node_info[i, 2]

In [60]:
recomb_val

np.float64(18.0)

In [65]:
# Recombination edge in node
node_index = int(node_info[i, 0])
recomb_idx = int(node_info[i, 2]) - 1
leaf_edge = int(recomb_edge[recomb_idx, 2]) - 1

tar_node_pos = np.where(recomb_node[leaf_edge] == node_info[i, 2])[0]

In [66]:
leaf_edge

2

In [67]:
tar_node_pos

array([0])

In [71]:
# Build ARG edges and track ancestral material
i = n  # Start after leaf nodes (0-indexed)
edge_index = 0

while i < node_max:
    recomb_val = node_info[i, 2]
    
    if recomb_val == 0:
        # Clonal tree node
        node_index = int(node_info[i, 0])
        leaf_edge = np.where(clonal_edge[:, 0] == node_index)[0]
        leaf_index = np.full(2, np.nan)
        leaf_node = np.full(2, np.nan)
        
        for le_idx in range(2):
            le = leaf_edge[le_idx]
            if len(recomb_node[le]) > 0:
                # Target node is last element
                tar_node = recomb_node[le][-1]
                leaf_index[le_idx] = np.where(node_info[:, 2] == tar_node)[0][0]
                leaf_node[le_idx] = node_info[int(leaf_index[le_idx]), 0]
            else:
                leaf_node[le_idx] = clonal_edge[le, 1]
                leaf_index[le_idx] = np.where(node_info[:, 0] == leaf_node[le_idx])[0][0]
        
        # Append edges
        edge_matrix[edge_index:edge_index+2, 0] = i
        edge_matrix[edge_index:edge_index+2, 1] = leaf_index
        edge_matrix[edge_index:edge_index+2, 2] = node_info[i, 1] - node_info[leaf_index.astype(int), 1]
        edge_mat_index[edge_index:edge_index+2] = leaf_index
        
        # Append root node material
        li0, li1 = int(leaf_index[0]), int(leaf_index[1])
        node_mat[i, :] = np.logical_or(
            np.nan_to_num(node_mat[li0, :], nan=0).astype(bool),
            np.nan_to_num(node_mat[li1, :], nan=0).astype(bool)
        )
        
        edge_index += 2
        i += 1
        
    elif recomb_val < 0:
        # Recombination edge out node
        node_index = node_info[i:i+2, 0].astype(int)
        recomb_idx = int(abs(node_info[i, 2])) - 1
        leaf_edge = int(recomb_edge[recomb_idx, 0]) - 1
        
        tar_node_pos = np.where(recomb_node[leaf_edge] == node_info[i, 2])[0]
        if len(tar_node_pos) > 0:
            tar_node_pos = tar_node_pos[0]
            if tar_node_pos == 0:
                leaf_node = clonal_edge[leaf_edge, 1]
            else:
                prev_node = recomb_node[leaf_edge][tar_node_pos]
                leaf_node = node_info[np.where(node_info[:, 2] == prev_node)[0][0], 0]
        else:
            leaf_node = clonal_edge[leaf_edge, 1]
        
        leaf_index = int(np.where(node_info[:, 0] == leaf_node)[0][0])
        
        # Append edges
        edge_matrix[edge_index:edge_index+2, 0] = [i, i + 1]
        edge_matrix[edge_index:edge_index+2, 1] = leaf_index
        edge_matrix[edge_index:edge_index+2, 2] = node_info[i, 1] - node_info[leaf_index, 1]
        edge_mat_index[edge_index:edge_index+2] = [i, i + 1]
        
        x = int(recomb_edge[recomb_idx, 4])
        y = int(recomb_edge[recomb_idx, 5])
        
        # Append root node material
        node_mat[i:i+2, :] = False
        # x:y in R is inclusive, in Python x-1:y is equivalent for 1-indexed to 0-indexed
        node_mat[i + 1, x-1:y] = np.nan_to_num(node_mat[leaf_index, x-1:y], nan=0).astype(bool)
        
        # For the complement (-(x:y) in R means all except x:y)
        mask = np.ones(2, dtype=bool)
        mask[x-1:y] = False
        node_mat[i, mask] = np.nan_to_num(node_mat[leaf_index, mask], nan=0).astype(bool)
        
        edge_index += 2
        i += 2
        
    elif recomb_val > 0:
        # Recombination edge in node
        node_index = int(node_info[i, 0])
        recomb_idx = int(node_info[i, 2]) - 1
        leaf_edge = int(recomb_edge[recomb_idx, 2]) - 1
        
        tar_node_pos = np.where(recomb_node[leaf_edge] == node_info[i, 2])[0]
        if len(tar_node_pos) > 0:
            tar_node_pos = tar_node_pos[0]
            if tar_node_pos == 0:
                if leaf_edge == 2 * n - 2:  # Root edge (0-indexed)
                    leaf_node = 2 * n - 2
                else:
                    leaf_node = clonal_edge[leaf_edge, 1]
            else:
                prev_node = recomb_node[leaf_edge][tar_node_pos]
                leaf_node = node_info[np.where(node_info[:, 2] == prev_node)[0][0], 0]
        else:
            leaf_node = clonal_edge[leaf_edge, 1]
        
        leaf_index = np.full(2, np.nan)
        leaf_index[0] = int(np.where(node_info[:, 0] == leaf_node)[0][0])
        
        # Find the corresponding out node
        neg_recomb = -node_info[i, 2]
        out_node_idx = np.where(node_info[:, 2] == neg_recomb)[0]
        if len(out_node_idx) > 0:
            leaf_index[1] = out_node_idx[0] + 1
        
        # Append edges
        edge_matrix[edge_index:edge_index+2, 0] = i
        edge_matrix[edge_index:edge_index+2, 1] = leaf_index
        edge_matrix[edge_index:edge_index+2, 2] = node_info[i, 1] - node_info[leaf_index.astype(int), 1]
        edge_mat_index[edge_index:edge_index+2] = leaf_index
        
        # Append root node material
        li0, li1 = int(leaf_index[0]), int(leaf_index[1])
        node_mat[i, :] = np.logical_or(
            np.nan_to_num(node_mat[li0, :], nan=0).astype(bool),
            np.nan_to_num(node_mat[li1, :], nan=0).astype(bool)
        )
        
        edge_index += 2
        i += 1
    else:
        # NaN case - skip
        i += 1

In [73]:
edge_matrix

array([[5.00000000e+00, 2.00000000e+00, 8.76779900e-03],
       [6.00000000e+00, 2.00000000e+00, 8.76779900e-03],
       [7.00000000e+00, 7.00000000e+00, 0.00000000e+00],
       [8.00000000e+00, 7.00000000e+00, 0.00000000e+00],
       [9.00000000e+00, 9.00000000e+00, 0.00000000e+00],
       [1.00000000e+01, 9.00000000e+00, 0.00000000e+00],
       [1.10000000e+01, 4.00000000e+00, 6.27433300e-02],
       [1.10000000e+01, 8.00000000e+00, 1.17682910e-02],
       [1.20000000e+01, 0.00000000e+00, 8.00621750e-02],
       [1.30000000e+01, 0.00000000e+00, 8.00621750e-02],
       [1.40000000e+01, 1.00000000e+00, 8.10807210e-02],
       [1.50000000e+01, 1.00000000e+00, 8.10807210e-02],
       [1.60000000e+01, 1.60000000e+01, 0.00000000e+00],
       [1.70000000e+01, 1.60000000e+01, 0.00000000e+00],
       [1.80000000e+01, 1.80000000e+01, 0.00000000e+00],
       [1.80000000e+01, 1.70000000e+01, 1.77357150e-02],
       [1.90000000e+01, 1.90000000e+01, 0.00000000e+00],
       [1.90000000e+01, 1.00000